In [1]:
%matplotlib inline

import itertools
import spacy
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras.backend as K
import tensorflow as tf

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Embedding, Dropout, Reshape, TimeDistributed
from keras.utils import to_categorical, plot_model

from keras_tqdm import TQDMNotebookCallback, TQDMCallback

from pprint import pprint

from scipy import stats
from sklearn.metrics import cohen_kappa_score, accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

from tqdm import tqdm

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 760 (CNMeM is disabled, cuDNN 5105)


In [2]:
TWITTER_DATA_FILEPATH = '/home/james/corpora/twitter/trec2011_with_hashtags.pkl'
TUMBLR_DATA_FILEPATH = '/home/james/corpora/tumblr/tumblr_halfday_tags_only.pkl'

In [3]:
# Email Settings
import smtplib

wherefrom = 'j.m.fiacco@gmail.com'
whereto = '6037323260@msg.fi.google.com'
gmail_pw = 'thdqhntcjnqglhxk'

def send_text(text='Hello World!'):
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login(wherefrom, gmail_pw)
        ret = server.sendmail(wherefrom, whereto, ' '.join(['[[EXPERIMENT]]', text]))
        server.quit()
        print ('Messsage sent.')
    except:
        print ('Message failed.')

# Data Processing

## Twitter

In [4]:
tumblr_data = pd.read_pickle(TUMBLR_DATA_FILEPATH)
tumblr_data

,post_id,activity_time_epoch,tumblog_id,post_title,post_short_url,post_type,post_caption,post_format,post_note_count,created_time_epoch,...,source_url,post_classifier,blog_classifier,accepts_answers,reblogged_from_post_id,reblogged_from_metadata,root_post_id,body,mentions,post_tags
0,26171566571,1.48887e+12,4.9315e+07,NaN,https://tmblr.co/ZsHqfvONycdh,text,NaN,html,1,1.34101e+12,...,NaN,safe,safe,False,NaN,[],-1,"<p>Started a <a href=""http://hpnetwork.tumblr....",{},"{(promo),(f4f),(p4p),(hogwarts)}"
1,36553508850,1.48887e+12,2.7401e+07,Esta es una carta que una hija le dejó a su mamá.,https://tmblr.co/ZhOQzwY2maVo,text,NaN,html,27040,1.35389e+12,...,http://thisbeautifulmelody.tumblr.com/post/127...,safe,safe,False,3.65503e+10,"[root_post_id#12755764180,root_post_blog_id#12...",1.27558e+10,"<p><a class=""tumblr_blog"" href=""http://frenill...",{},"{(fav),(asdf),(favoritos),(historia)}"
2,41719816100,1.48887e+12,4.9315e+07,NaN,https://tmblr.co/ZsHqfvcsiT_a,text,NaN,html,6,1.3594e+12,...,http://zoiekravitz.tumblr.com/post/26012912538...,safe,safe,False,3.03554e+10,"[root_post_id#26012912538,root_post_blog_id#49...",2.60129e+10,"<p><a href=""http://freed-from-azkaban.tumblr.c...",{},"{(botw),(botm),(blog rate),(rate),(triwizard),..."
8,49986293281,1.48886e+12,1.46435e+07,NaN,https://tmblr.co/Zv-4kxkZQauX,text,NaN,html,111726,1.36807e+12,...,http://imagineyouricon.tumblr.com/post/4972798...,safe,safe,False,4.99858e+10,"[root_post_id#49727982949,root_post_blog_id#92...",4.9728e+10,"<p><a class=""tumblr_blog"" href=""http://maskums...",{},"{(DEAD),(vg comics),(world of tumblr),(vg rela..."
9,50046391893,1.48886e+12,2.09521e+07,NaN,https://tmblr.co/Zup0Mxkc-rPL,text,NaN,html,9,1.36814e+12,...,NaN,safe,safe,False,NaN,[],-1,<p>Never thought I&rsquo;d say it&hellip;but I...,{},"{(jolex),(alex and izzie),(grey's anatomy),(jo..."
10,50627731464,1.48886e+12,2.09521e+07,NaN,https://tmblr.co/Zup0Mxl9fU08,text,NaN,html,1,1.36876e+12,...,http://braddocksgirl14.tumblr.com/,safe,safe,False,NaN,[],-1,<p>Can anyone get me a good quality screen sho...,{},"{(jolex),(grey's anatomy),(otp: i'm gonna marr..."
11,52887718490,1.48889e+12,8.76784e+07,3959,https://tmblr.co/ZkEUNtnGMefQ,text,NaN,html,0,1.37115e+12,...,NaN,safe,safe,False,NaN,[],-1,"<a href=""http://68.media.tumblr.com/4115772c54...",{},{(Beautiful)}
13,61036656360,1.48887e+12,3.6539e+07,NaN,https://tmblr.co/Z2RZQwus4NRe,text,NaN,html,238369,1.37901e+12,...,http://perchu.tumblr.com/post/61024362215/fini...,safe,safe,False,6.10308e+10,"[root_post_id#61024362215,root_post_blog_id#63...",6.10244e+10,"<p><a class=""tumblr_blog"" href=""http://groupin...",{},"{(holy shit),(incredible),(The Lorax),(good po..."
16,64056001821,1.48886e+12,1.3273e+08,Luna lunera.,https://tmblr.co/ZENdhqxg2GKT,text,NaN,html,1,1.38179e+12,...,NaN,safe,safe,False,NaN,[],-1,<p>… pero siempre estoy acá… y mi amor sigue i...,{},{(versolibre)}
17,65616082669,1.48885e+12,3.52205e+07,NaN,https://tmblr.co/ZE2Kxtz71VRj,text,NaN,html,0,1.38323e+12,...,NaN,safe,safe,False,NaN,[],-1,"<p>It was December 21st, Howl’s eighth birthda...",{},{(writing)}


In [5]:
tumblr_data = tumblr_data[(tumblr_data['post_tags'] != '{}')&(tumblr_data['post_tags'].notnull())&(tumblr_data['body'].notnull())]
len(tumblr_data)

786306

In [33]:
with open('/home/james/corpora/tumblr/tumblr_halfday_tags_only.pkl', 'wb') as f:
    pickle.dump(tumblr_data, f)

In [6]:
tumblr_text = tumblr_data['body']
tumblr_tags = tumblr_data['post_tags'].apply(lambda t: [str(s.strip('()')) for s in t.strip('{}').split(',')])
len(tumblr_text), len(tumblr_tags)

(786306, 786306)

In [35]:
test_tag = list(tumblr_tags)[8567]
test_tag

['Ben Carson']

In [8]:
tumblr_lengths = tumblr_text.apply(lambda s: len(s))
print ('Tumblr Max Length (characters):', np.max(tumblr_lengths))
print ('Tumblr 99th Percentile:', np.percentile(tumblr_lengths, 99))
print ('Tumblr 140 character percentile:', '{:.3f}'.format(stats.percentileofscore(tumblr_lengths, 140)))
print ('Tumblr 200 character percentile:', '{:.3f}'.format(stats.percentileofscore(tumblr_lengths, 200)))
num_tags = tumblr_tags.apply(lambda t: len(t))
print ('Tumblr Max Tags:', np.max(num_tags))

Tumblr Max Length (characters): 266100
Tumblr 99th Percentile: 15446.85
Tumblr 140 character percentile: 12.865
Tumblr 200 character percentile: 18.402
Tumblr Max Tags: 30


In [9]:
# Extract character set

chars = set()
for text in tqdm(tumblr_text):
    chars.update(set(text))
    
chars = sorted(chars)
chars = ['<PAD>'] + chars
print ('{} unique characters.'.format(len(chars)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

12915 unique characters.


In [10]:
MAX_LENGTH = 200

In [11]:
# Vectorize tweets

tumblr_X = np.zeros((len(tumblr_text), MAX_LENGTH), dtype=np.bool)
for i, text in tqdm(enumerate(tumblr_text), total=len(tumblr_text)):
    for t, char in enumerate(text):
        if t >= MAX_LENGTH:
            break
        tumblr_X[i, t] = char_indices[char]

100%|██████████| 786306/786306 [00:34<00:00, 23066.03it/s]


In [12]:
# Extract tag set

tag_set = set()
for t in tqdm(tumblr_tags):
    tag_set.update(set(t))
    
tag_set = sorted(tag_set)
print ('{} unique tags.'.format(len(tag_set)))

tag_indices = dict((t, i) for i, t in enumerate(tag_set))
indices_tag = dict((i, t) for i, t in enumerate(tag_set))

100%|██████████| 786306/786306 [00:00<00:00, 833650.71it/s]


707622 unique tags.


In [14]:
# Vectorize tags

tumblr_y = np.zeros((len(tumblr_tags), len(tag_set)), dtype=np.bool)
for i, tags in tqdm(enumerate(tumblr_tags), total=len(tumblr_tags)):
    for tag in tags:
        tumblr_y[i, tag_indices[tag]] = 1

MemoryError: 

In [15]:
sss = ShuffleSplit(n_splits=1, random_state=12345)

# Create X and y matricies
for train_idx, test_idx in sss.split(twitter_X, twitter_y):
    train_X, test_X = twitter_X[train_idx], twitter_X[test_idx]
    train_y, test_y = twitter_y[train_idx], twitter_y[test_idx]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

NameError: name 'twitter_X' is not defined

# Build Model

## BiLSTM Baseline

In [13]:
model = Sequential()
model.add(Embedding(len(chars) + 1,
                    64,
                    input_length=MAX_LENGTH,
                    trainable=True,
                    mask_zero=True))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(.5))
model.add(Dense(len(tag_set), activation='softmax'))
print (model.output_shape)


(None, 748)


In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [15]:
filepath="./weights.twitter.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_categorical_accuracy', patience=3)
callbacks_list = [checkpoint, early_stop]

In [16]:
model.fit(train_X,
              train_y,
              validation_data=(test_X, test_y),
              shuffle=True,
              batch_size=32,
              epochs=10,
              verbose=0,
              callbacks=callbacks_list)

send_text('BiLSTM Baseline Training complete!')

Epoch 00000: val_categorical_accuracy improved from -inf to 0.16518, saving model to ./weights.twitter.best.hdf5
Epoch 00001: val_categorical_accuracy did not improve
Epoch 00002: val_categorical_accuracy improved from 0.16518 to 0.17534, saving model to ./weights.twitter.best.hdf5
Epoch 00003: val_categorical_accuracy improved from 0.17534 to 0.17878, saving model to ./weights.twitter.best.hdf5
Epoch 00004: val_categorical_accuracy did not improve
Epoch 00005: val_categorical_accuracy did not improve
Epoch 00006: val_categorical_accuracy improved from 0.17878 to 0.18272, saving model to ./weights.twitter.best.hdf5
Epoch 00007: val_categorical_accuracy did not improve
Epoch 00008: val_categorical_accuracy did not improve
Epoch 00009: val_categorical_accuracy did not improve
Messsage sent.
